In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 64.1 MB/s eta 0:00:00


In [37]:
import gensim
import os
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from sklearn.decomposition import PCA
import plotly.express as px
from nltk.corpus import stopwords

In [5]:
print(os.listdir('data'))

['.ipynb_checkpoints', '5 - A Dance with Dragons.txt', '4 - A Feast for Crows.txt', '3 - A Storm of Swords.txt', '2 - A Clash of Kings.txt', '1 - A Game of Thrones.txt']


In [6]:
rm -rf data/.ipynb_checkpoints/

In [41]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
# story = []
# for filename in os.listdir('data'):
#     f = open(os.path.join('data',filename), encoding='utf-8', errors='ignore')
#     corpus = f.read()
#     raw_sent = sent_tokenize(corpus)
#     for sent in raw_sent:
#         story.append(simple_preprocess(sent))


stop_words = set(stopwords.words("english"))

story = []
for filename in os.listdir('data'):
    with open(os.path.join('data', filename), encoding='utf-8', errors='ignore') as f:
        corpus = f.read()
        raw_sent = sent_tokenize(corpus)
        for sent in raw_sent:
            tokens = simple_preprocess(sent)
            # remove stopwords
            tokens = [word for word in tokens if word not in stop_words]
            story.append(tokens)

In [43]:
story

[['george',
  'martin',
  'dance',
  'dragons',
  'book',
  'five',
  'song',
  'ice',
  'fire',
  'dedication',
  'one',
  'fans',
  'lodey',
  'trebla',
  'stego',
  'pod',
  'caress',
  'yags',
  'ray',
  'mr',
  'kate',
  'chataya',
  'mormont',
  'mich',
  'jamie',
  'vanessa',
  'ro',
  'stubby',
  'louise',
  'agravaine',
  'wert',
  'malt',
  'jo',
  'mouse',
  'telisiane',
  'blackfyre',
  'bronn',
  'stone',
  'coyotes',
  'daughter',
  'rest',
  'madmen',
  'wild',
  'women',
  'brotherhood',
  'without',
  'banners',
  'website',
  'wizards',
  'elio',
  'linda',
  'lords',
  'westeros',
  'winter',
  'fabio',
  'wic',
  'gibbs',
  'dragonstone',
  'started',
  'men',
  'women',
  'asshai',
  'spain',
  'sang',
  'us',
  'bear',
  'maiden',
  'fair',
  'fabulous',
  'fans',
  'italy',
  'gave',
  'much',
  'wine',
  'readers',
  'finland',
  'germany',
  'brazil',
  'portugal',
  'france',
  'netherlands',
  'distant',
  'lands',
  'youve',
  'waiting',
  'dance',
  'friend

In [44]:
len(story)

158874

In [45]:
model = gensim.models.Word2Vec(
    window=10, # no of words before and after before 10 and after 10
    min_count=2, # minimum length of sentence
)

In [46]:
# build vocab

model.build_vocab(story)

In [47]:
#  train model

model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(4416952, 4596315)

🔹 1. 4416952 → work_done

This is the number of effective words (or “word instances”) the model actually trained on.
It can be slightly less than the total words due to:

subsampling

removing rare words

negative sampling efficiency

internal threading behavior

🔹 2. 4596315 → total_words

This is the total number of words expected to be processed based on:

total_examples × epochs × sentence_length


It is the “target workload” the model aims to cover.

🧠 Why they differ?

Because Word2Vec internally removes some words from training.

Examples:

words with frequency below min_count

randomly removed words due to subsampling (sample=1e-3)

threading batch drop

So work_done < total_words is normal.

✔️ In summary
Number	Meaning
6575164	Effective words the model actually trained on
8625265	Expected total words for all epochs

You can ignore these numbers unless debugging training efficiency.

In [48]:
model.wv.most_similar('john')

[('anne', 0.9828627109527588),
 ('butchers', 0.9812894463539124),
 ('valuables', 0.9795833826065063),
 ('using', 0.9794633388519287),
 ('sloey', 0.9794485569000244),
 ('ding', 0.9788118004798889),
 ('groell', 0.9787222146987915),
 ('aggar', 0.9780687093734741),
 ('un', 0.9779543876647949),
 ('voyager', 0.9779216051101685)]

In [49]:
model.wv.most_similar('daenerys')

[('stormborn', 0.9380800724029541),
 ('unburnt', 0.9280085563659668),
 ('targaryen', 0.8684925436973572),
 ('elia', 0.8547031283378601),
 ('myrcella', 0.843086838722229),
 ('dorne', 0.8399031758308411),
 ('princess', 0.8329973220825195),
 ('queen', 0.8166594505310059),
 ('regent', 0.8165208101272583),
 ('dornes', 0.8093420267105103)]

In [50]:
model.wv.doesnt_match(['jon','rikon','robb','arya','sansa','bran'])

'jon'

In [51]:
model.wv.doesnt_match(['cersei','jaime','tywin','tyrion','joffrey'])

'tywin'

In [52]:
model.wv['queen']

array([ 0.8816608 ,  0.54939896, -0.12092335,  1.275508  ,  1.0892581 ,
       -0.79662186, -1.4121175 ,  1.9080428 , -0.74535793, -0.98784727,
       -0.17794597, -1.131814  ,  0.24641913,  0.8659723 , -0.17589113,
       -0.07731815, -0.14188813, -0.71263975, -0.01522631, -1.6281202 ,
        0.958644  ,  1.0701146 ,  0.56295264, -0.9987486 ,  0.2644509 ,
       -1.2206165 , -0.589327  , -0.8499843 , -0.32346106,  0.2693359 ,
       -0.09988669, -0.22694656,  0.88940537,  0.8690132 ,  0.7193106 ,
        1.4171613 , -0.29382536,  0.03148571,  0.95834637, -0.43340096,
        0.8562487 , -0.6545241 ,  0.05592192,  1.1714292 ,  0.21666983,
       -0.06662467, -0.3704932 , -0.7369208 , -0.05294483,  0.13789548,
        1.04834   ,  0.59874225, -0.5268367 ,  1.1709346 ,  0.0375572 ,
        0.32866243, -0.28101426, -1.6937882 , -0.39403126, -0.27508432,
        1.5764326 ,  1.4488369 , -0.5617931 , -1.3788266 , -1.4155786 ,
        0.8658246 , -0.3100413 ,  0.05003645, -0.11777166,  0.48

In [53]:
model.wv.similarity('arya','sansa')

np.float32(0.8213266)

In [54]:
model.wv.similarity('stark','lannister')

np.float32(0.5511507)

In [55]:
model.wv.similarity('arya','tywin')

np.float32(0.05986006)

In [56]:
model.wv.get_normed_vectors().shape

(17726, 100)

In [57]:
#  for visualizing and plotting this vector
#  using PCA, for dimentionality reduction
#  17869, unique words
#  100 dimention
#  model.wv.get_normed_vectors() with this we can get all the vectors

In [58]:
#  words


words = model.wv.index_to_key
print(len(words))

17726


In [59]:
words

['said',
 'lord',
 'would',
 'one',
 'ser',
 'could',
 'man',
 'men',
 'back',
 'king',
 'well',
 'like',
 'jon',
 'old',
 'hand',
 'even',
 'never',
 'tyrion',
 'see',
 'know',
 'made',
 'father',
 'eyes',
 'black',
 'told',
 'thought',
 'lady',
 'time',
 'long',
 'might',
 'us',
 'come',
 'still',
 'face',
 'head',
 'red',
 'way',
 'page',
 'boy',
 'must',
 'good',
 'two',
 'little',
 'brother',
 'took',
 'came',
 'though',
 'say',
 'night',
 'three',
 'away',
 'queen',
 'dead',
 'son',
 'blood',
 'take',
 'go',
 'half',
 'make',
 'arya',
 'saw',
 'white',
 'day',
 'first',
 'jaime',
 'look',
 'want',
 'much',
 'enough',
 'tell',
 'sword',
 'great',
 'looked',
 'bran',
 'girl',
 'left',
 'knew',
 'asked',
 'gave',
 'called',
 'wall',
 'every',
 'heard',
 'maester',
 'yet',
 'went',
 'let',
 'sansa',
 'turned',
 'need',
 'behind',
 'dany',
 'around',
 'another',
 'beneath',
 'across',
 'snow',
 'keep',
 'gods',
 'found',
 'knight',
 'woman',
 'gold',
 'last',
 'grace',
 'castle',
 'th

In [60]:
pca = PCA(n_components=3)

X = pca.fit_transform(model.wv.get_normed_vectors())

In [61]:
X.shape

(17726, 3)

In [64]:
fig = px.scatter_3d(X[:100], x=0, y=1, z=2, color=words[:100])
fig.show()

#  plotted top 100 words from 17726 words